In [1]:
import os
import urllib
import re
from urllib import request
import json
import requests
import xml.etree.ElementTree as ET
import pickle

In [2]:
####base url and apikey
#### it requires an apiKey from Elsevier
#doi = '10.1016/j.enggeo.2012.07.017'
apiKey = 'replace with Elsevier apiKey'
base = 'https://api.elsevier.com/content/article/doi/'

In [3]:
def retrieveTextPaper(url, apikey, doi):
    params = {
        'APIKey': apikey,
        'view': 'FULL',
        'httpAccept': 'text/xml'  #### application/json    text/xml
        }
    post_args = urllib.parse.urlencode(params)

    test = request.urlopen(url+doi+'?'+post_args)
    html = test.read()
    save()

    html = ET.fromstring(html)
    paragraphs = []

    for n in html:
        if 'originalText' in n.tag:
            OT = n
            break

    for n in OT:
        if 'doc' in n.tag:
            doc = n
            break

    for n in doc:
        if 'serial' in n.tag:
            serial = n
            break

    for n in serial:
        if 'article' in n.tag:
            article = n
            break

    for n in article:
        if 'body' in n.tag:
            body = n
            break

    for n in body:
        if 'sections' in n.tag:
            sections = n
            break

    for n in sections:
        for m in n:
            if 'para' in m.tag:
                # TODO: Remove refs, tables, and eq.
                paragraphs.append(''.join(m.itertext()))

    text1 = ''.join(paragraphs)
    return text1

In [4]:
####finding dois of papers to use

##base url
base2 = 'https://api.elsevier.com/content/search/scidir'

def getDOIs(url, query, apikey, subscribed='true', counts = 200, year = None):
    dois = []
    if year is None:
        for n in range(1, 5801,200):
            try:
                if n == 1:
                    param2 = {
                        'query': '{} AND pub-date IS {}'.format(query, year),
                        'APIKey': apikey,
                        'subscribed': subscribed,
                        'count': counts
                    }
                else:
                    param2 = {
                        'query': '{} AND pub-date IS {}'.format(query, year),
                        'APIKey': apikey,
                        'subscribed': subscribed,
                        'count': counts,
                        'start': str(n)
                    }
                post_args2 = urllib.parse.urlencode(param2)

                newTest = request.urlopen(base2+'?'+post_args2)
                html = newTest.read()
                for z in json.loads(html.decode("utf-8"))['search-results']['entry']:
                    dois.append(z['dc:identifier'][4:])
            except:
                continue


    else:
        year = year
        param2 = {
            'query': '{} AND pub-date IS {}'.format(query, year),
            'APIKey': apikey,
            'subscribed': subscribed,
            'count': counts
            }
        post_args2 = urllib.parse.urlencode(param2)
        newTest = request.urlopen(base2+'?'+post_args2)
        html = newTest.read()
        la = json.loads(html.decode("utf-8"))['search-results']
        number = int(la['opensearch:totalResults'])
        print(year, number)
        for n in range(1,number,200):
            if n == 1:
                param2 = {
                    'query': '{} AND pub-date IS {}'.format(query, year),
                    'APIKey': apikey,
                    'subscribed': subscribed,
                    'count': counts
                    }
            else:
                param2 = {
                    'query': '{} AND pub-date IS {}'.format(query, year),
                    'APIKey': apikey,
                    'subscribed': subscribed,
                    'count': counts,
                    'start': n
                    }
            post_args2 = urllib.parse.urlencode(param2)
            newTest = request.urlopen(base2+'?'+post_args2)
            html = newTest.read()
            for z in json.loads(html.decode("utf-8"))['search-results']['entry']:
                dois.append(z['dc:identifier'][4:])

    return dois

In [5]:
####to get all DOIs from 2000, to 2018 using the keyword lithology
allDOI = []
for n in range(2000, 2019, 1):
    DOIs = getDOIs(base2, 'lithology', apiKey, year = n)
    allDOI.append(DOIs)

allDOI = [item for sublist in allDOI for item in sublist]
len(allDOI)

2000 796
2001 834
2002 910
2003 974
2004 965
2005 1052
2006 1276
2007 1262
2008 1413
2009 1611
2010 1604
2011 1580
2012 1925
2013 2185
2014 2445
2015 2683
2016 2909
2017 3279
2018 2060


31744

In [ ]:
##saving DOI list as picke object
with open('allDOI.pickle', 'wb') as f:
    pickle.dump(allDOI, f)
##importing DOI list saved
with open('allDOI.pickle', 'rb') as f:
    tmp = pickle.load(f)

In [ ]:
##to download all papers from DOIs list as txt files into the paper folder
##The number of papers is over 30,000, so it takes a while to finish!

for n in tmp:
    try:
        doi = n
        content = retrieveTextPaper(url = base, apikey = apiKey, doi=doi)
        if len(content) != 0:
            with open('papers/{}.txt'.format(doi.replace('/', '-')), 'w') as files:
                files.write(content)
                files.close
    except:
        continue